# Libraries

In [1]:
import requests
import sys
from podcast_ad_skipper.google_cloud import *
from podcast_ad_skipper.data_preparation import *
from podcast_ad_skipper.params import *
#Libraries:
import pandas as pd
import numpy as np
import os
import time
import librosa
import librosa.display
import matplotlib.pyplot as plt
from pydub import AudioSegment
from scipy.ndimage import zoom
from keras.applications import VGG16
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import cv2 as cv
from keras import layers, models, Model
from keras.optimizers import Adam
import tensorflow as tf
from pydub import AudioSegment
import pickle


#   DATA FROM BQ

In [3]:
bq_client = auth_gc_bigquery()

Running in local environment.
Authenticated successfully with BigQuery! ✅


In [4]:
table_id = f"{GCP_PROJECT_ID}.Numpy_Arrays_Dataset.processed_train_data_II"
table_id

'podcast-ad-skipper.Numpy_Arrays_Dataset.processed_train_data_II'

In [5]:
%%time
output= get_output_query_bigquery(bq_client, table_id, custom="Leo")
output

Query returned 3209 results
CPU times: user 68.6 ms, sys: 9.38 ms, total: 78 ms
Wall time: 17.8 s


In [6]:
%%time
processed_output = get_bq_processed_data(output)
processed_output

CPU times: user 35.1 s, sys: 6.9 s, total: 42 s
Wall time: 9min 30s


([array([[-40.03315735, -46.07726288, -80.        , ..., -48.34635162,
          -40.48298645, -37.88743591],
         [-39.62623215, -45.77576447, -72.32196045, ..., -50.55674744,
          -44.52566528, -38.23501205],
         [-38.80467987, -44.81970596, -64.58843231, ..., -49.43707275,
          -45.82523727, -36.46101761],
         ...,
         [-71.52801514, -63.03861237, -57.53601074, ..., -36.90007782,
          -52.37269592, -61.06830215],
         [-73.36112976, -70.35489655, -62.58636856, ..., -43.34296799,
          -55.45136261, -60.44869232],
         [-72.67279053, -69.42549133, -65.45346069, ..., -47.10861206,
          -61.96922684, -61.12409973]]),
  array([[-44.89272308, -51.58975983, -80.        , ..., -75.39718628,
          -43.84510422, -32.2700882 ],
         [-44.0619278 , -50.74234772, -80.        , ..., -74.57250977,
          -44.21195602, -32.16596222],
         [-42.84126282, -49.37010193, -72.90988159, ..., -67.6204071 ,
          -42.6558609 , -31.29141

In [7]:
processed_output[0]

[array([[-40.03315735, -46.07726288, -80.        , ..., -48.34635162,
         -40.48298645, -37.88743591],
        [-39.62623215, -45.77576447, -72.32196045, ..., -50.55674744,
         -44.52566528, -38.23501205],
        [-38.80467987, -44.81970596, -64.58843231, ..., -49.43707275,
         -45.82523727, -36.46101761],
        ...,
        [-71.52801514, -63.03861237, -57.53601074, ..., -36.90007782,
         -52.37269592, -61.06830215],
        [-73.36112976, -70.35489655, -62.58636856, ..., -43.34296799,
         -55.45136261, -60.44869232],
        [-72.67279053, -69.42549133, -65.45346069, ..., -47.10861206,
         -61.96922684, -61.12409973]]),
 array([[-44.89272308, -51.58975983, -80.        , ..., -75.39718628,
         -43.84510422, -32.2700882 ],
        [-44.0619278 , -50.74234772, -80.        , ..., -74.57250977,
         -44.21195602, -32.16596222],
        [-42.84126282, -49.37010193, -72.90988159, ..., -67.6204071 ,
         -42.6558609 , -31.29141045],
        ...,


In [8]:
processed_output[1]

[0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [9]:
spectrogram_np = np.array(processed_output[0])
labels_np = np.array(processed_output[1])
print(spectrogram_np.shape)
print(labels_np.shape)

(3209, 128, 216)
(3209,)


# MODEL

In [10]:
# Define X and y:
X = spectrogram_np
y = labels_np

# Print the shape of X and y
print(X.shape)
print(y.shape)

(3209, 128, 216)
(3209,)


In [11]:
X = np.expand_dims(X, axis=-1)
X.shape

(3209, 128, 216, 1)

In [13]:
# Split the data into training and testing sets: 80% training, 20% testing:
X_train, X_test, y_train, y_test = train_test_split(
    np.array(X),
    y, test_size=0.2,
    random_state=42,
    stratify=y # This will ensure that the distribution of labels is the same in both training and testing sets
    )

In [14]:
model = tf.keras.models.Sequential([
    # Input layer - note that Input() doesn't take an activation
    tf.keras.layers.Input(shape=(128, 216, 1)),
    # Conv2D layer
    tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'),
    # Flatten layer
    tf.keras.layers.Flatten(),
    # Dense layers
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile model
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train model for 10 epochs, capture the history
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping,])

Epoch 1/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 11s 127ms/step - accuracy: 0.5086 - loss: 470.6979 - val_accuracy: 0.5327 - val_loss: 0.6929
Epoch 2/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 10s 126ms/step - accuracy: 0.5258 - loss: 0.9481 - val_accuracy: 0.5327 - val_loss: 0.6925
Epoch 3/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 10s 120ms/step - accuracy: 0.5317 - loss: 0.6925 - val_accuracy: 0.5327 - val_loss: 0.6922
Epoch 4/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 10s 119ms/step - accuracy: 0.5180 - loss: 0.6926 - val_accuracy: 0.5327 - val_loss: 0.6918
Epoch 5/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 10s 120ms/step - accuracy: 0.5385 - loss: 0.6915 - val_accuracy: 0.5327 - val_loss: 0.6916
Epoch 6/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 10s 122ms/step - accuracy: 0.5335 - loss: 0.6915 - val_accuracy: 0.5327 - val_loss: 0.6914
Epoch 7/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 10s 122ms/step - accuracy: 0.5290 - loss: 0.6917 - val_accuracy: 0.5327 - val_loss: 0.6913
Epoch 8/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 10s 123ms/step - accuracy: 0.5290 - loss: 0.6916 - val_ac

In [15]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 64, 108, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 221184)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │    56,623,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 169,871,813 (648.01 MB)

 Trainable params: 56,623,937 (216.00 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 113,247,876 (432.01 MB)

In [16]:
# Evaluate the model:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5397 - loss: 0.6901
Test accuracy: 0.5327102541923523


In [17]:
X_predict = np.expand_dims(create_spectrogram('../raw_data/5_sec_clips/glucosegoddess/0_1_6030_glucosegoddess.wav'), axis=-1)
X_predict_2 = np.expand_dims(create_spectrogram('../raw_data/5_sec_clips/trevornoah/0_1_9615_trevornoah.wav'), axis=-1)
X_predict_3 = np.expand_dims(create_spectrogram('../raw_data/5_sec_clips/glucosegoddess/1_4636_6030_glucosegoddess.wav'), axis=-1)
X_predict_4 = np.expand_dims(create_spectrogram('../raw_data/5_sec_clips/0_1006_5864_billionairepersonalitydisorder.wav'), axis=-1)
print(X_predict.shape, X_predict_2.shape, X_predict_3.shape, X_predict_4.shape)
processed_input = X_predict.reshape(1, 128, 216, 1)
processed_input_2 = X_predict_2.reshape(1, 128, 216, 1)
processed_input_3 = X_predict_3.reshape(1, 128, 216, 1)
processed_input_4 = X_predict_4.reshape(1, 128, 216, 1)
print(processed_input.shape, processed_input_2.shape, processed_input_3.shape, processed_input_4.shape)

(128, 216, 1) (128, 216, 1) (128, 216, 1) (128, 216, 1)
(1, 128, 216, 1) (1, 128, 216, 1) (1, 128, 216, 1) (1, 128, 216, 1)


In [18]:
predition = model.predict(processed_input)
predition_2 = model.predict(processed_input_2)
predition_3 = model.predict(processed_input_3)
predition_4 = model.predict(processed_input_4)
predition, predition_2, predition_3, predition_4

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


(array([[0.53259915]], dtype=float32),
 array([[0.53259915]], dtype=float32),
 array([[0.53259915]], dtype=float32),
 array([[0.53259915]], dtype=float32))

In [ ]:
def create_spectrogram(wav_path, sr=22050):
    y, sr = librosa.load(wav_path, sr=sr)
    # Create mel spectrogram
    mel_spect = librosa.feature.melspectrogram(
        y=y,
        sr=sr,
        n_mels=128,  # Number of mel bands
        fmax=8000)    # Maximum frequency
    # Convert to log scale and return
    return np.array(librosa.power_to_db(mel_spect, ref=np.max))

In [ ]:
# Function to detect ads within a podcast
def detect_ads(podcast_file, model, clip_duration=5):
    """
    This function splits the podcast into clips, creates spectrograms, and uses the model to detect ads.
    podcast_file: Path to the podcast audio file (mp3)
    model: The trained model for ad detection
    clip_duration: Duration of each clip in seconds (default 5)
    return: List of ad segments (start_time, end_time) in seconds

    EXPLANATION:
    ongoing_ad Flag: Tracks whether an ad sequence is currently in progress.
    ad_start_time Variable: Stores the start time of the ongoing ad segment.
    Conditional Check for Consecutive Ads: If an ad prediction is followed by another ad, it extends the ongoing ad segment.
    If the following clip is not an ad, it finalizes the current ad segment and appends it to ad_segments.
    Final Check After Loop: If the podcast ends while an ad segment is ongoing, it appends this last ad segment to ad_segments.
    This way, the function will detect ad segments lasting longer than one clip duration and merge consecutive ad clips into single segments.
    Adjust the 0.5 threshold in if prediction[0][0] > 0.5 as needed based on model performance.
    """
    podcast = AudioSegment.from_file(podcast_file)  # Load podcast file
    podcast_duration = len(podcast) / 1000  # Duration in seconds
    print(f"Podcast duration: {podcast_duration} seconds.")

    ad_segments = []  # Store detected ad segments
    ongoing_ad = False  # Track if an ad segment is ongoing
    ad_start_time = 0   # Start time of the ongoing ad segment
    previous_prediction = 0  # Track previous clip prediction (0 = no ad, 1 = ad)

    # Process the podcast in chunks
    for i in range(0, int(podcast_duration), clip_duration):
        start_time = i * 1000  # Convert to milliseconds
        end_time = (i + clip_duration) * 1000
        clip = podcast[start_time:end_time]

        # Save the clip as a temporary wav file for processing
        clip_file = "temp_clip.wav"
        clip.export(clip_file, format="wav")

        # Create and validate spectrogram
        spectrogram = create_spectrogram(clip_file)
        if spectrogram.shape == (128, 216):
            spectrogram = np.expand_dims(spectrogram, axis=-1)  # Add channel dimension
            spectrogram = np.expand_dims(spectrogram, axis=0)   # Add batch dimension
        else:
            print(f"Clip starting at {start_time / 1000} seconds has incorrect shape: {spectrogram.shape}")
            os.remove(clip_file)
            continue  # Skip this clip if shape is incorrect

        # Predict using the model
        prediction = model.predict(spectrogram)
        current_prediction = int(prediction[0][0] > 0.5)  # Convert to binary (0 or 1)
        print(f"Prediction for clip starting at {start_time / 1000} seconds: {prediction}")

        # Check if the previous clip was an ad
        if current_prediction == 1: # Current clip detected as an ad
            if previous_prediction == 0: # Previous clip was not an ad
                ad_start_time = i # Start a new ad sequence
            ongoing_ad = True

        elif current_prediction == 0 and previous_prediction == 1:
            # End of an ad sequence when the current clip is not an ad but the previous was
            ad_segments.append((ad_start_time, i))  # Record the ad segment
            ongoing_ad = False

        # Update previous prediction to the current one for the next iteration
        previous_prediction = current_prediction

        # Remove the temporary file
        os.remove(clip_file)

    # Append any remaining ad segment at the end of the podcast
    if ongoing_ad:
        ad_segments.append((ad_start_time, int(podcast_duration)))

    print(f"Total ad segments detected: {len(ad_segments)}")
    return ad_segments